# ESG Stock Performance Analysis - Exploratory Data Analysis

**Project:** Evaluating the Relationship Between ESG Performance, Stock Returns, and Carbon Footprints  
**Course:** DATA 512  
**Author:** Ayush Mall

This notebook provides exploratory data analysis of the ESG and stock performance data.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## Load Data

In [ ]:
# Load analysis dataset
df = pd.read_csv('../data/final/analysis_dataset.csv')

print(f"Dataset shape: {df.shape}")
print(f"Number of companies: {len(df)}")
print(f"Number of variables: {len(df.columns)}")

In [ ]:
# View first few rows
df.head()

In [ ]:
# Column names
print("Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

## Descriptive Statistics

In [ ]:
# Select key numeric columns
key_cols = [col for col in df.columns if any(k in col.lower() for k in 
            ['esg', 'sharpe', 'volatility', 'beta', 'return', 'market_cap'])]
key_cols = [col for col in key_cols if 'sector' not in col.lower()]

# Descriptive statistics
df[key_cols].describe().round(4)

## Missing Values Analysis

In [ ]:
# Check missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if len(missing) > 0:
    print("Missing values:")
    for col, count in missing.items():
        pct = (count / len(df)) * 100
        print(f"  {col}: {count} ({pct:.1f}%)")
else:
    print("✅ No missing values!")

## ESG Score Distribution

In [ ]:
# Find ESG column
esg_col = [col for col in df.columns if 'esg' in col.lower() and 'score' in col.lower()]
esg_col = [col for col in esg_col if not any(x in col.lower() for x in ['environment', 'social', 'governance'])]

if esg_col:
    esg_col = esg_col[0]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(df[esg_col].dropna(), bins=30, edgecolor='black', alpha=0.7)
    ax.axvline(df[esg_col].mean(), color='red', linestyle='--', label=f'Mean: {df[esg_col].mean():.2f}')
    ax.set_xlabel('ESG Score')
    ax.set_ylabel('Frequency')
    ax.set_title('Distribution of ESG Scores')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.show()
else:
    print("ESG score column not found")

## Performance Metrics Distribution

In [ ]:
# Sharpe Ratio distribution
if 'Sharpe_Ratio' in df.columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    ax1.hist(df['Sharpe_Ratio'].dropna(), bins=30, edgecolor='black', alpha=0.7)
    ax1.axvline(df['Sharpe_Ratio'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["Sharpe_Ratio"].mean():.2f}')
    ax1.set_xlabel('Sharpe Ratio')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Sharpe Ratios')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Box plot
    ax2.boxplot(df['Sharpe_Ratio'].dropna(), vert=False)
    ax2.set_xlabel('Sharpe Ratio')
    ax2.set_title('Sharpe Ratio Box Plot')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Volatility distribution
if 'Volatility' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.hist(df['Volatility'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='green')
    ax.axvline(df['Volatility'].mean(), color='red', linestyle='--', 
               label=f'Mean: {df["Volatility"].mean():.2%}')
    ax.set_xlabel('Annualized Volatility')
    ax.set_ylabel('Frequency')
    ax.set_title('Distribution of Stock Volatility')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    plt.show()

## Correlation Analysis

In [ ]:
# Correlation matrix
corr_cols = [col for col in key_cols if df[col].dtype in ['float64', 'int64']]
corr_matrix = df[corr_cols].corr()

# Plot heatmap
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, ax=ax, cbar_kws={'shrink': 0.8})
ax.set_title('Correlation Matrix of Key Variables')
plt.tight_layout()
plt.show()

## Sector Analysis

In [ ]:
# Sector distribution
if 'Sector' in df.columns:
    sector_counts = df['Sector'].value_counts()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    sector_counts.plot(kind='barh', ax=ax)
    ax.set_xlabel('Number of Companies')
    ax.set_ylabel('Sector')
    ax.set_title('Distribution of Companies by Sector')
    ax.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()

## ESG vs. Performance Relationships

In [ ]:
# ESG vs Sharpe Ratio scatter plot
if esg_col and 'Sharpe_Ratio' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Scatter plot
    ax.scatter(df[esg_col], df['Sharpe_Ratio'], alpha=0.5)
    
    # Add regression line
    plot_df = df[[esg_col, 'Sharpe_Ratio']].dropna()
    z = np.polyfit(plot_df[esg_col], plot_df['Sharpe_Ratio'], 1)
    p = np.poly1d(z)
    ax.plot(plot_df[esg_col], p(plot_df[esg_col]), "r--", alpha=0.8, label='Best fit')
    
    ax.set_xlabel('ESG Score')
    ax.set_ylabel('Sharpe Ratio')
    ax.set_title('ESG Score vs. Risk-Adjusted Returns')
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.show()
    
    # Correlation
    corr = df[[esg_col, 'Sharpe_Ratio']].corr().iloc[0, 1]
    print(f"Correlation: {corr:.4f}")

In [ ]:
# ESG vs Volatility scatter plot
if esg_col and 'Volatility' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.scatter(df[esg_col], df['Volatility'], alpha=0.5, color='green')
    
    # Add regression line
    plot_df = df[[esg_col, 'Volatility']].dropna()
    z = np.polyfit(plot_df[esg_col], plot_df['Volatility'], 1)
    p = np.poly1d(z)
    ax.plot(plot_df[esg_col], p(plot_df[esg_col]), "r--", alpha=0.8, label='Best fit')
    
    ax.set_xlabel('ESG Score')
    ax.set_ylabel('Volatility (Annualized)')
    ax.set_title('ESG Score vs. Stock Volatility')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    ax.grid(True, alpha=0.3)
    ax.legend()
    plt.show()
    
    # Correlation
    corr = df[[esg_col, 'Volatility']].corr().iloc[0, 1]
    print(f"Correlation: {corr:.4f}")

## Summary Statistics by ESG Quartile

In [ ]:
# Create ESG quartiles
if esg_col:
    df['ESG_Quartile'] = pd.qcut(df[esg_col], q=4, labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4 (High)'])
    
    # Summary statistics by quartile
    summary_vars = ['Sharpe_Ratio', 'Volatility', 'Annualized_Return']
    summary_vars = [v for v in summary_vars if v in df.columns]
    
    if summary_vars:
        quartile_summary = df.groupby('ESG_Quartile')[summary_vars].mean()
        print("\nMean Performance by ESG Quartile:")
        print(quartile_summary.round(4))

## Conclusion

This exploratory analysis provides initial insights into:
1. Distribution of ESG scores across companies
2. Distribution of performance metrics (Sharpe ratio, volatility)
3. Correlations between ESG and performance variables
4. Sector differences in ESG scores

Next steps:
- Run formal regression analysis (RQ1, RQ2, RQ3)
- Check regression diagnostics
- Create publication-quality visualizations